# Start Simulation

In [1]:
# env.clean(clean_all=True)

In [2]:
import random
import numpy as np
import os
import time
from PIL import Image

import torch
torch.__version__
device = torch.device("cuda")

In [3]:
import getpass
user = getpass.getuser()
print(user)

yizhou


In [4]:
usd_path = f"omniverse://localhost/Users/{user}/UVA/uva_table.usd"

In [5]:
from omni.isaac.kit import SimulationApp    
simulation_app = SimulationApp({"headless": True, "open_usd": usd_path,  "livesync_usd": usd_path}) 

Starting kit application with the fillowing args:  ['/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/exts', '--ext-folder', '/home/yizhou/.local/share/ov/pkg/isaac_sim-2022.1.1/apps', '--/physics/cudaDevice=0', '--portable', '--no-window']
Passing the following args to the base kit application:  ['-f', '/home/yizhou/.local/share/jupyter/runtime/kernel-f8597301-a7bc-

OmniAssetFileFormat


Warp initialized:
   Version: 0.2.2
   CUDA device: NVIDIA GeForce RTX 3090
   Kernel cache: /home/yizhou/.cache/warp/0.2.2
[2.515s] [ext: omni.kit.menu.create-1.0.2] startup
[2.516s] [ext: omni.mdl-0.1.0] startup
[2.542s] [ext: omni.kit.window.file_importer-1.0.4] startup
[2.543s] [ext: omni.kit.window.file_exporter-1.0.4] startup
[2.545s] [ext: omni.kit.material.library-1.3.10] startup
[2.549s] [ext: omni.kit.window.drop_support-1.0.0] startup
[2.550s] [ext: omni.kit.window.file-1.3.16] startup
[2.554s] [ext: omni.kit.context_menu-1.3.9] startup
[2.558s] [ext: omni.kit.window.property-1.6.3] startup
[2.559s] [ext: omni.kit.window.content_browser-2.4.28] startup
[2.572s] [ext: omni.kit.widget.stage-2.6.15] startup
[2.580s] [ext: omni.isaac.version-1.0.0] startup
[2.581s] [ext: omni.kit.property.usd-3.14.9] startup
[2.668s] [ext: omni.kit.viewport.legacy_gizmos-1.0.0] startup
[2.672s] [ext: omni.hydra.rtx-0.1.0] startup
[2.697s] [ext: omni.renderer-rtx-0.0.0] startup
[2.700s] [ext: omn

In [6]:
# set log level
import logging
import carb

logging.getLogger("omni.hydra").setLevel(logging.ERROR)
logging.getLogger("omni.isaac.urdf").setLevel(logging.ERROR)
logging.getLogger("omni.physx.plugin").setLevel(logging.ERROR)

logging.getLogger("omni.isaac.synthetic_utils").setLevel(logging.ERROR)
logging.getLogger("omni.isaac.synthetic_utils.syntheticdata").setLevel(logging.ERROR)
logging.getLogger("omni.hydra.scene_delegate.plugin").setLevel(logging.ERROR)


l = carb.logging.LEVEL_ERROR
carb.settings.get_settings().set("/log/level", l)
carb.settings.get_settings().set("/log/fileLogLevel", l)
carb.settings.get_settings().set("/log/outputStreamLevel", l)

# # This logged error is printed as it should
# carb.log_error("ERROR")
# # This warning is printed but should not
# carb.log_warn("WARNING")

# Config

In [7]:
from task.config import DATA_PATH, FEATURE_PATH
task_type = "Table"
side_choice = "Border"
base_asset_id = 0
load_nucleus = True

In [8]:
pause

NameError: name 'pause' is not defined

# Init Env

In [9]:
from uva_env import UvaEnv
env = UvaEnv()

In [10]:
from task.utils import add_scene_default
add_scene_default()

In [13]:
print(list(env.stage.TraverseAll()))

[Usd.Prim(</physicsScene>), Usd.Prim(</World>), Usd.Prim(</World/defaultLight>), Usd.Prim(</World/physicsScene>), Usd.Prim(</OmniverseKit_Persp>), Usd.Prim(</OmniverseKit_Front>), Usd.Prim(</OmniverseKit_Top>), Usd.Prim(</OmniverseKit_Right>), Usd.Prim(</Render>), Usd.Prim(</Render/RenderProduct_Viewport>), Usd.Prim(</Render/Vars>), Usd.Prim(</Render/Vars/LdrColor>)]


In [12]:
env.clean()
env.world.step(render=True)

# Scene

In [14]:
from task.scene import ArrangeScene
scene = ArrangeScene(task_type, side_choice, base_asset_id = 0, traj_id = 0, load_nucleus = load_nucleus)
env.scene = scene

In [15]:
# add base
scene.add_base_asset()

asset_prim: omniverse://localhost/Users/yizhou/Asset/T/Appleseed/Appleseed_CoffeeTable.usd


In [16]:
# add room
scene.add_room()

asset_prim: omniverse://localhost/Users/yizhou/Asset/S/0/layout.usd


In [17]:
env.world.step(render=True)

In [18]:
# randomize scene
from layout.randomizer import Randomizer
randomizer = Randomizer(load_nucleus = load_nucleus)

# Reward

In [19]:
from uv.reward import Rewarder

In [20]:
rewarder = Rewarder(env.world)
env.rewarder = rewarder

# Render

In [21]:
from render.helper import RenderHelper
render = RenderHelper(task_type, side_choice)

render.add_task_cameras(camera_type="main")
render.add_task_cameras(camera_type="side")
render.set_cameras()

In [22]:
render.camera_paths

['/World/render/camera_main', '/World/render/camera_side']

In [23]:
pause

NameError: name 'pause' is not defined

#  Learning

In [24]:
# from learning.network.resnet import ResNetFeatureExtractor

# feature extraction
from learning.utils import extract_image_clip_feature_and_save, obtain_action_from_trainer
from transformers import CLIPProcessor, CLIPModel

feature_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
feature_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# replay buffer
import json
from learning.replay_buffer import ReplayBuffer
from learning.config import *

buffer = ReplayBuffer(max_size=2000)

# trainer
from learning.network.sac import *

policy = Policy()

qf1 = QFunction()
qf2 = QFunction()
target_qf1 = QFunction()
target_qf2 = QFunction()

from learning.sac_trainer import SACTrainer

trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
     use_automatic_entropy_tuning = False, 
     policy_lr=1e-3, 
     qf_lr=1e-3,
     target_update_period = 5)

# Trajectory

In [25]:
use_network = True
debug = True

In [26]:
total_traj = 0
total_step = 0

---------------------

In [27]:
# traj config
for traj_id in range(100):
    print("total_traj: ", total_traj)
    env.scene.traj_id = traj_id
    
    # base
    if total_traj % 5 == 0:
        asset_count = len(env.scene.base_asset_file_paths)
        base_asset_id = random.choice([_ for _ in range(asset_count)])
        env.scene.base_asset_id = base_asset_id    
        env.clean(clean_base = True, clean_all = False)
        scene.add_base_asset()
        
        # randomize scene
        randomizer.randomize_house(rand = True)
        
    image_folder = os.path.join(DATA_PATH, task_type, side_choice, str(traj_id))
    
    env.world.step(render = True)
    env.world.step(render = True)

    # get images
    env.world.render()
    images = render.get_images()
    render.save_rgb(images[0]['rgb'], image_folder, "0")
    render.save_rgb(images[1]['rgb'], image_folder, "0_side")
    
    ## extract feature
    
#     extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
#         feature_extractor, os.path.join(image_folder, str(0) + ".pt"))
    if use_network:
        extract_image_clip_feature_and_save(f"{image_folder}/{0}.png", feature_model, feature_processor, 
            f"{image_folder}/{0}.pt",)
    

    # trajectory
    for step in range(5):
        total_step += 1
        
        # sample an object
        env.add_scene_obj(mode = "random")
        
        # TODO: get action from sampling
        if not use_network or total_traj < 10 or np.random.rand() < 0.2:
            x, y = np.tanh(np.random.randn()), np.tanh(np.random.randn())
        else:
            image_feature_file = f"{image_folder}/{step}.pt"
            
            object_info = env.scene.objects[-1]
            object_type = object_info["type"]
            obj_name = object_info["name"][:-4]
            object_feature_file = os.path.join(FEATURE_PATH, object_type, obj_name + ".pt")
            x, y = obtain_action_from_trainer(image_feature_file, object_feature_file, trainer, 
                                              scaler=np.exp(- total_traj / 100))
            object_info["use_network"] = True
        
        # load the object into the scene
        env.put_last_object((x, y)) 
        env.world.step(render=True)
        
        # register the object to the world for physics update
        env.register_last_object()
        env.world.step(render=True)

        # get images
        env.world.render()
        images = render.get_images()
        render.save_rgb(images[0]['rgb'], image_folder, str(step + 1))
        render.save_rgb(images[1]['rgb'], image_folder, f"{step + 1}_side")

        ## calculate reward
        env.calculate_last_reward(simulation_step = 30)
        
        ## extract feature
#         extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
#             feature_extractor, os.path.join(image_folder, str(step + 1) + ".pt"))
        if use_network:
            extract_image_clip_feature_and_save(f"{image_folder}/{step + 1}.png", feature_model, feature_processor, 
                f"{image_folder}/{step + 1}.pt",)
    

        ## reset
        env.world.reset()
        env.world.step(render=True)
        
        ## trainer nework
        if use_network and total_step % UPDATE_TRAINER_STEPS == 0 and total_traj > 5:
            batch = buffer.sample_batch(batch_size = BATCH_SIZE)
            trainer.update(batch)
            
            if debug and total_step % 10 == 0:
                rewards = batch['rewards'].to(device)
                terminals = batch['terminals'].to(device)
                obs = batch['observations'].to(device)
                actions = batch['actions'].to(device)
                next_obs = batch['next_observations'].to(device)
                obj_features = batch['object_features'].to(device)
                
                dist = trainer.policy(obs, obj_features)
                pred = trainer.qf1(obs, obj_features, actions)
                print(f"debug {total_traj}/{total_step}", #"\n dist: ", dist.mean.flatten().tolist(), dist.stddev.flatten().tolist(),
                      "\n pred:", pred.flatten().tolist(),
                      "\n rewards: ", rewards.flatten().tolist())


    # Record
    record = env.scene.get_scene_data()
    env.scene.save_scene_data()
    # print("record: ", record)
    
    # Add record to buffer
    if use_network:
        buffer.add_scene_sample(record)

    # Reset (env clean)
    env.clean(clean_all = False)
    env.step(render = True)
    total_traj += 1

total_traj:  0
asset_prim: omniverse://localhost/Users/yizhou/Asset/T/Jenning/Jennings_Table.usd
1.2] startup
[9.649s] [ext: omni.graph.window.core-1.23.4] startup
[9.659s] [ext: omni.graph.instancing-1.1.4] startup
[9.668s] [ext: omni.kit.menu.edit-1.0.6] startup
[9.670s] [ext: omni.isaac.isaac_sensor-1.0.2] startup
2023-01-23 18:48:21 [9,668ms] [Warning] [omni.physx.plugin] Deprecated: getSimulationEventStream is deprecated, please use getSimulationEventStreamV2
[9.690s] [ext: omni.graph.tutorials-1.1.2] startup
[9.714s] [ext: omni.graph.window.action-1.3.8] startup
[9.716s] [ext: omni.kit.widget.live-0.1.0] startup
[9.722s] [ext: omni.kit.widget.layers-1.5.17] startup
[9.740s] [ext: omni.graph.bundle.action-1.0.0] startup
[9.740s] [ext: omni.isaac.range_sensor-0.4.3] startup
[9.772s] [ext: omni.kit.property.layer-1.1.2] startup
[9.774s] [ext: omni.replicator.isaac-1.3.2] startup
[9.892s] [ext: omni.isaac.kit-0.2.1] startup
[9.892s] [ext: omni.graph.window.generic-1.3.8] startup
[9.8

2023-01-23 18:49:59 [107,789ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/GadgetsAndTool

2023-01-23 18:49:59 [108,188ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-23 18:49:59 [108,292ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-23 18:50:01 [109,656ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Tchotchke_01

2023-01-23 18:50:01 [109,656ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Tchotchke_01

2023-01-23 18:50:01 [109,656ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Tchotchke_01

2023-01-23 18:50:01 [109,656ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objec

2023-01-23 18:50:03 [112,045ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Tchotchke_02

2023-01-23 18:50:04 [112,913ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-23 18:50:04 [113,017ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-23 18:50:06 [114,419ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Tchotchke_01

2023-01-23 18:50:06 [114,419ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Tchotchke_01

2023-01-23 18:50:06 [114,419ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Tchotchke_01

2023-01-23 18:50:06 [114,419ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects

2023-01-23 18:50:10 [119,209ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Flatware

2023-01-23 18:50:11 [119,541ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-23 18:50:11 [119,644ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-23 18:50:12 [121,047ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware

2023-01-23 18:50:12 [121,047ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware

2023-01-23 18:50:12 [121,047ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware

2023-01-23 18:50:12 [121,047ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerwar

2023-01-23 18:50:14 [123,362ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Tchotchke

2023-01-23 18:50:15 [123,779ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-23 18:50:15 [123,883ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-23 18:50:16 [125,354ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware

2023-01-23 18:50:16 [125,354ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware

2023-01-23 18:50:16 [125,354ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware

2023-01-23 18:50:16 [125,354ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerwa

...

asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Flatware/Fork_Small.usd
2023-01-23 19:12:26 [1,454,947ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Flatware_01

2023-01-23 19:12:26 [1,455,185ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-23 19:12:26 [1,455,290ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-23 19:12:28 [1,457,084ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Flatware

2023-01-23 19:12:28 [1,457,084ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Flatware_01

2023-01-23 19:12:28 [1,457,084ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Vase

2023-01-23 19:12:28 [1,457,085ms] [Warning] [omni.hydr

total_traj:  99
2023-01-23 19:12:29 [1,457,825ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-23 19:12:29 [1,457,929ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Dinnerware/P_Glassware_Tall.usd
2023-01-23 19:12:31 [1,459,706ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Dinnerware

2023-01-23 19:12:31 [1,459,899ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-23 19:12:31 [1,460,002ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-23 19:12:33 [1,461,722ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware

2023-01-23 19:12:33 [1,461,722

2023-01-23 19:12:35 [1,464,127ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware

2023-01-23 19:12:35 [1,464,127ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine

2023-01-23 19:12:35 [1,464,127ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware

2023-01-23 19:12:35 [1,464,127ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine

2023-01-23 19:12:35 [1,464,127ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware

2023-01-23 19:12:35 [1,464,127ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine

2023-01-23 19:12:35 [1,464,127ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/obje

asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Tchotchke/PumpkinSmall.usd
2023-01-23 19:12:38 [1,466,729ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Tchotchke

2023-01-23 19:12:38 [1,467,034ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-23 19:12:38 [1,467,138ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-23 19:12:40 [1,468,932ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware

2023-01-23 19:12:40 [1,468,932ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine

2023-01-23 19:12:40 [1,468,932ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Dinnerware

2023-01-23 19:12:40 [1,468,932ms] [Warning] [omn

debug 99/500 
 pred: [1.2382421493530273, 1.6455190181732178, 1.1493682861328125, 2.2133100032806396, 1.6870675086975098, 1.471451759338379, 1.9399937391281128, 0.981056272983551] 
 rewards:  [-1.5125658512115479, -0.4306640625, -0.4306182861328125, 0.9799346923828125, 0.8546115159988403, -0.43062591552734375, 0.9149320721626282, -0.4306640625]


# Debug

In [ ]:
env.clean(clean_base = True, clean_all = False)

In [ ]:
# get images
env.world.render()
env.world.render()
images = render.get_images()

Image.fromarray(images[0]['rgb'], "RGBA").resize((300, 300))

##  Debug Training

In [ ]:
import json
from learning.replay_buffer import ReplayBuffer

buffer = ReplayBuffer(max_size=1000)

for i in range(100):
    replay = json.load(open(f"{DATA_PATH}/{task_type}/{side_choice}/{i}/scene.json"))
    buffer.add_scene_sample(replay)

In [ ]:
len(buffer.dataset)

In [ ]:
# trainer
from learning.network.sac import *

policy = Policy()

qf1 = QFunction()
qf2 = QFunction()

target_qf1 = QFunction()
target_qf1.eval()

target_qf2 = QFunction()
target_qf2.eval()

from learning.sac_trainer import SACTrainer

trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
     use_automatic_entropy_tuning = True, 
     policy_lr=1e-3, 
     qf_lr=1e-3,
     target_update_period = 5)

In [ ]:
self = trainer

In [ ]:
for _ in range(500):
    batch = buffer.sample_batch(batch_size=6)

    rewards = batch['rewards'].to(device)
    terminals = batch['terminals'].to(device)
    obs = batch['observations'].to(device)
    actions = batch['actions'].to(device)
    next_obs = batch['next_observations'].to(device)

    obj_features = batch['object_features'].to(device)

    # print("rewards", rewards)

    self.update(batch)
    
    if _ % 10 == 0:
        dist = self.policy(obs, obj_features)
        pred = self.qf1(obs, obj_features, actions)
        print(_, "\n dist   : ", [round(x, 3) for x in dist.mean.flatten().tolist()], 
                  [round(x, 3) for x in dist.stddev.flatten().tolist()],
              "\n pred   :", [round(x, 3) for x in pred.flatten().tolist()],
              "\n rewards: ", [round(x, 3) for x in rewards.flatten().tolist()])
    
#     # debug
#     pred = self.qf1(obs, obj_features, actions)
#     loss = self.qf_criterion(pred, rewards)
    
#     self.qf1_optimizer.zero_grad()
#     loss.backward()
#     self.qf1_optimizer.step()
    
#     if _ % 20 == 0:
#         print(_, "\n debug loss: ", loss.item(), "\n pred:", 
#               pred.flatten().tolist(), "\n rewards: ", rewards.flatten().tolist())

In [ ]:
dist = self.policy(obs.to(device), obj_features.to(device))

In [ ]:
dist.normal_mean, dist.stddev

In [ ]:
dist.sample(scaler = 0.2)

In [ ]:
self = dist

In [ ]:
scaler = 0.1

In [ ]:
self.normal_mean + scaler * self.normal_std * MultivariateDiagonalNormal(
    torch.zeros(self.normal_mean.size()).to(self.device),
    torch.ones(self.normal_std.size()).to(self.device)
).sample()

# distribution

In [ ]:
from learning.distributions import MultivariateDiagonalNormal

In [ ]:
dist = JustNormal(dist.mean, dist.stddev)

In [ ]:
dist.rsample_and_logprob()